# Homework 10

## Problem 1: Adding Points on Elliptic Curves

*You cannot use Sage's own implementation of elliptic curves in this problem!*

Given the coefficients `vE = [a, b]` of an elliptic curve (so, the curve is given by the equation $y^2 = x^3 + ax + b$) and a pair of points $P$, $Q$, compute the sum $P+Q$.  Here we will use the convention that the point infinity $\mathcal{O}$ is given by $0$ (zero), in both input and output.

**Your output should be a list `[x,y]`, not a tuple `(x,y)`!**

*You do not need to check that the points are on the curve!*  We will also assume that $a$, $b$ and the coordinates of $P$ and $Q$ are all in the same field.

**Hint:** The algorithm is [given in the book](https://luisfinotti.org/pcimc/16-Elliptic_Curves.html#al-sum).

In [ ]:
def add_EC(vE, P, Q):
    """
    Given points P and Q in an elliptic curve given by vE = [a, b], give the
    sum of the points.

    INPUTS:
    vE: a pair of elements [a, b] with -16 * (4 * a^3 + 26 * b^2) not zero and
         giving the coefficients of the Weirestrass equation y^2 = x^3 + a*x + b
         of the curve;
    P: a point in the curve: either a pair of elements satisfying the equation
       or 0, for the point at infinity;
    Q: another point (perhaps the same as P).

    OUTPUT:
    The result of P + Q, so another point on the curve (a pair or 0).
    """
    ...

Let's test it:

In [ ]:
def random_EC_gen(p):
    F = FiniteField(p)
    a = 0
    b = 0
    while -16 * (4*a^3 + 27*b^2) == 0:
        a = F.random_element()
        b = F. random_element()
    return [a, b]

number_of_tests = 30
min_b = 30
max_b = 40
failed = False

for _ in range(number_of_tests):
    # create EC
    p = random_prime(2^max_b, lbound=2^min_b)
    vE = random_EC_gen(p)
    E = EllipticCurve(vE)

    # random point != O
    PP = E.random_element()
    while PP == E(0):
        PP = E.random_element()
    P = [PP[0], PP[1]]

    # Q = 0
    result = add_EC(vE, P, 0)
    expected = P
    if result != expected:
        print("Failed test 1")
        failed = True
        Q = 0
        break

    # P = 0
    result = add_EC(vE, 0, P)
    expected = P
    if result != expected:
        print("Failed test 2")
        failed = True
        Q = P
        P = 0
        break

    # Q = -P
    result = add_EC(vE, P, [P[0], -P[1]])
    expected = 0
    if result != expected:
        print("Failed test 3")
        failed = True
        Q = [P[0], -P[1]]
        break

    # Q = P
    RR = PP + PP
    expected = [RR[0], RR[1]]
    result = add_EC(vE, P, P)
    if result != expected:
        print("Failed test 4")
        failed = True
        Q = P
        break

    # generate new random point != P and O
    QQ = E.random_element()
    while QQ == PP or QQ == E(0) or QQ == -PP:
        QQ = E.random_element()
    Q = [QQ[0], QQ[1]]

    # Q != P, -P, O
    RR = PP + QQ
    expected = [RR[0], RR[1]]
    result = add_EC(vE, P, Q)
    if result != expected:
        print("Failed test 5")
        failed = True
        break

    # if there is a point of order 2, test it!
    PR.<x> = PolynomialRing(GF(p))
    f = x^3 + vE[0]*x + vE[1]
    v = f.roots()
    if len(v) != 0:  # if there is P with 2*P = O
        x1 = v[0][0]
        result = add_EC(vE, [x1, 0], [x1, 0])
        expected = 0
        if result != expected:
            print("Failed test 6")
            failed = True
            P = Q = [x1, 0]

if failed:
    print(f"For {p = }, {vE = }, {P = }, {Q = }.  It should return {expected}, but it returned {result}.")
else:
    print("It works!")

## Problem 2: Ternary Expansion

Given a positive integer $n$, give its *ternary expansion* as [explained in the book](https://luisfinotti.org/pcimc/16-Elliptic_Curves.html#ternary-expansion), i.e., express it as $n = e_0 + e_1 \cdot 2 + e_2 \cdot 2^2 + \cdots + e_k \cdot 2^k$, where $e_i \in \{0,1,-1\}$, $e_k = 1$, and at most $\lceil (k+1)/2\rceil$ of them are not $0$.

Your output should be simply the list/vector `[e0, e1, e2,..., ek]`.

**Hints:** 
- To get the digits in base $2$ of `n`, you can use `ZZ(n).bits()`.
- The algorithm is [described in the book](https://luisfinotti.org/pcimc/16-Elliptic_Curves.html#al:ternary).


In [ ]:
def tern_exp(n):
    """
    Gives the ternary expansion of a positive integer n.

    INPUT:
    n: a positive integer.

    OUTPUT:
    A list of 0's, 1's and -1's, with no consecutive non-zero elements, ending
    with 1, and when multiplies by the powers of two (from 0) add to n.
    """
    ternary = ZZ(n).bits()
    ...

Let's test it.  Here is the example from class:

In [ ]:
tern_exp(3545) == [1, 0, 0, -1, 0, -1, 0, 0, 0, -1, 0, 0, 1]

More tests:

In [ ]:
def consecutive_non_zero(v):
    """
    Tests if v has consecutive non-zero entries.

    INPUT:
    v: a list of numbers.

    OUTPUT:
    True if there no non-zero consecutive numbers, False otherwise.
    """
    length = len(v)
    i = 0
    while True:
        while i < length and v[i] == 0:
            i += 1
        if i >= length - 1:
            return True
        if v[i + 1] != 0:
            return False
        i += 1


number_of_tests = 20
min_n = 2000
max_n = 5000

for _ in range(number_of_tests):
    n = randint(min_n, max_n)
    e = tern_exp(n)
    k = len(e)

    if e[-1] != 1:
        print(f"For {n = }, your expansion {e} had the last digit {e[-1]}, while it must be 1.")
        break

    if not set(e).issubset({0, 1, -1}):
        print(f"Your expansion {e}, contained numbers other than 0, 1, -1.")
        break

    if not consecutive_non_zero(e):
        print(f"For n = {0}, your expansion {e} had consecutive non-zero entries, which is not allowed.")
        break

    nn = sum(e[i]*2^i for i in range(k))
    if nn != n:
        print(f"For {n = }, your expansion {e} adds to {nn}, not n.")
        break

else:
    print("It works!")

## Problem 3: Menezes-Vanstone Elliptic Curve ElGamal

In this problem we implement the [Menezes-Vanstone Elliptic Curve ElGamal cryptosystem](https://luisfinotti.org/pcimc/16-Elliptic_Curves.html#menezes-vanstone-elgamal-cryptography).  The book also has [an example](https://luisfinotti.org/pcimc/16-Elliptic_Curves.html#id1).

We will assume we have an elliptic curve $E/\mathbb{F}_p$ and a point $P \in E(\mathbb{F}_p)$ of order $M$ chosen.

### 3(a): Encryption

Given $P$ (in $E(\mathbb{F}_p)$) of order $M$, a public key $Q$ (denoted $Q_A$ in the book) and a message consisting of $m_1, m_2 \in \mathbb{F}_p$, this function should produce the encrypted message $(R,c_1,c_2)$ as [described in the book](https://luisfinotti.org/pcimc/16-Elliptic_Curves.html#menezes-vanstone-elgamal-cryptography) (so $R \in E(\mathbb{F}_p)$, $c_1, c_2 \in \mathbb{F}_p$).

**Note:** Chose the random integer $n_B$ (in the algorithm) between $2$ and $M-1$.  ($M$ should be known, so it is one of the arguments of the function.)

**Hints:** 
- To check if some point $P$ is equal to $\mathcal{O}$ in Sage, you can do `P.is_zero()`.
- To check if either coordinate of $P \neq \mathcal{O}$ is zero, you can do `0 in P.xy()`.

In [ ]:
def MV_EC_enc(P, M, Q, m1, m2):
    """
    Menezes-Vanstone EC ElGamal encryption.

    INPUTS:
    P: a point in some elliptic curve over a finite field;
    M: the order of the point P;
    Q: the public ecnryption key;
    m1, m2: the message (in the base field of the curve).

    OUTPUT:
    The encoded message, a triple (R, c1, c2), where R is a point on the same
    curve, and c1, c2 are on the base field of the curve.
    """
    ...

### 3(b): Decryption

Given $E$, $P$, a private key $n$ (denoted $n_A$ in the text), and cyphertext `v = (R, c1, c2)` (as above), it should output the original message `[m1, m2]`.

In [ ]:
def MV_EC_dec(P, n, v):
    """
    Menezes-Vanstone EC ElGamal decryption.

    INPUTS:
    P: a point in some elliptic curve over a finite field;
    n: the private ecnryption key;
    v = [R, c1, c2]: an encrypted message (R in the same curve as P and c1 and
                     c2 in the base field of the curve).

    OUTPUT:
    The decoded message, i.e., a pair [m1, m2] with both entries in the base field
    of the elliptic curve.
    """
    ...

Let's now test both.  (We will use the same elliptic curve as used for bitcoin ECDSA.)

In [ ]:
p = 2^256 - 2^32 - 977
F = GF(p)
E = EllipticCurve([F(0), F(7)])

number_of_tests = 30

for _ in range(number_of_tests):
    P = E.random_element()
    M = P.order()
    n = randint(2, M - 1)
    Q = n * P
    m1 = F(randint(2, p - 2))
    m2 = F(randint(2, p - 2))
    v = MV_EC_enc(P, M, Q, m1, m2)
    m = MV_EC_dec(P, n, v)
    if m != [m1, m2]:
        print(f"The original message was {[m1, m2]}, but the decryption gave {m}.")
        break
else:
    print("It works!")

## Problem 4: ECDSA

In this problem we implement the ECDSA, as [described in the book](https://luisfinotti.org/pcimc/16-Elliptic_Curves.html#elliptic-curve-dsa).  We assume we have some elliptic curve $E/\mathbb{F}_p$ and a point $G$ of large (known) prime order $q$.

### 4(a): Signature

Given $G \in E(\mathbb{F}_p)$, $q$ (both as above), a document $D \in \mathbb{F}_q$, and a (secret) signing key $s$, it should return $(D,s_1,s_2)$ (with $(s_1,s_2) \in \mathbb{F}_q^2$ being the signature).

**Notes:** 
- Even though $|G|=q$, Sage does *not* allow you to multiply $G$ by an element in $\mathbb{F}_q = \mathbb{Z}/q\mathbb{Z}$.  You can only multiply $G$ by integers.  So, if $a \in \mathbb{F}_q$, to compute $a \cdot G$, you need to do `ZZ(a) * G`, not `a * G`.
- Again, to convert an element from $\mathbb{F}_p$ to $\mathbb{F}_q$, it is safer to first convert it to an *integer*.

In [ ]:
def ECDSA_sign(G, q, D, s):
    """
    ECDSA signing algorithm.

    INPUTS:
    G: a point on an elliptic curve over a finite field;
    q: a prime, order of G;
    D: a document to be signed (in Zmod(q));
    s: the private signing key.

    OUTPUT:
    A triple (D, s1, s2), where D is the original document and (s1, s2) its
    signature, with both entries in Zmod(q) as well.
    """
    ...

### 4(b): Verification

Now, given the point $G$, a verification key $V$, the signed document $v = (D,s_1,s_2)$, your function should return `True` if it can be verified, and `False` otherwise.

**Notes:**

- Again, even though $|G|=q$, Sage does *not* allow you to multiply $G$ by an element in $\mathbb{F}_q = \mathbb{Z}/q\mathbb{Z}$.  You can only multiply $G$ by integers.  So, if $a \in \mathbb{F}_q$, to compute $a \cdot G$, you need to do `ZZ(a) * G`, not `a * G`.
- To get the field of definition $\mathbb{F}_q$ from $s_1$, we can do `Fq = parent(s1)`.

In [ ]:
def ECDSA_ver(G, V, v):
    """
    ECDSA verification algorithm.
    """
    ...

Again, let's test both:

In [ ]:
p = 2^256 - 2^32 - 977
F = GF(p)
E = EllipticCurve([F(0), F(7)])
q = E.count_points()

number_of_tests = 30

for _ in range(number_of_tests):
    G = E.random_element()
    s = randint(2, q - 1)
    V = s * G
    D = randint(2, q - 1)

    signed_D = ECDSA_sign(G, q, D, s)

    if not ECDSA_ver(G, V, signed_D):
        print("Signature was valid, but failed to authenticate.")
        break

    DD = D + randint(2, q - 1)
    faked_D = [DD, signed_D[1], signed_D[2]]

    if ECDSA_ver(G, V, faked_D):
        print("Document was altered, but passed authentication.")
        break

    Fq = FiniteField(q)
    faked_sign = [D, signed_D[1] + Fq(randint(2, q - 1)), signed_D[2] + Fq(randint(2, q - 1))]
    if ECDSA_ver(G, V, faked_sign):
        print("Signature was fake, but passed authentication.")
        break
else:
    print("It works!")